# Visualization of results

Load modules and constants

In [1]:
import shutil
import os
import src.utils as utils
from src.data.dataloader import mtlDataModule 

from src.data.manifests import generate_manifest

import torchvision.transforms as T
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
from PIL import Image
from pathlib import Path

from src.utils import load_yaml
from src.models.model_loader import ModelLoader
import torch

ROOT_DIR = "/zhome/3b/d/154066/repos/multitask-mayhem"

os.chdir(ROOT_DIR)

CLASS_LOOKUP = load_yaml(ROOT_DIR+"/configs/class_lookup.yaml")

/zhome/3b/d/154066/miniconda3/envs/multitask-mayhem/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Get config to initialize `pl.DataModule` and create dataloaders

In [4]:
config_file = ROOT_DIR + "/models/ssdlite_baseline_22-12-04T181106/ssdlite_baseline_22-12-04T181106.yaml"


data_module = mtlDataModule(config_path=config_file)

seg, det = False, False
if data_module.config["model"] in ["fasterrcnn", "fasterrcnn_mobilenetv3", "ssdlite"]:
    det = True
elif data_module.config["model"] in ["deeplabv3"]:
    seg = True

data_module.config["batch_size"] = 1
data_module.config["num_workers"] = 0
data_module.config["shuffle"] = False


model_loader = ModelLoader(config=data_module.config)
model = model_loader.grab_model()
model_folder = str(Path(config_file).parents[0])
model.load_state_dict(torch.load(model_folder+"/weights/best.pth", map_location=torch.device('cpu')))

data_module.prepare_data()
data_module.setup(stage="fit")
train_dataloader = data_module.train_dataloader()
data_module.setup(stage="validate")
valid_dataloader = data_module.val_dataloader()
data_module.setup(stage="test")
test_dataloader = data_module.test_dataloader()

100%|██████████| 8/8 [00:12<00:00,  1.51s/it]


Prepare landing folder and label names

In [12]:
test_inference = ROOT_DIR+"/notebooks/test_inference"

if os.path.exists(test_inference):
    shutil.rmtree(test_inference)

os.makedirs(test_inference, exist_ok=True)

In [13]:
model.eval()
for i, batch in enumerate(test_dataloader):
    image, targets = batch

    targets = model(image)
    
    if det:
        boxes = targets[0]["boxes"]
        labels = targets[0]["labels"]
        scores = targets[0]["scores"]
        score_mask = scores > 0.5

        boxes_filtered = boxes[score_mask]
        labels_filtered = labels[score_mask]
        scores_filtered = scores[score_mask]


        label_names = [CLASS_LOOKUP["bbox_rev"][label.item()] for label in labels_filtered]
    elif seg:
        masks = targets[0]

    
    img = image[0].mul(255).type(torch.uint8)

    if det:
        drawn_image = draw_bounding_boxes(img, boxes_filtered, label_names)
    elif seg:
        drawn_image = draw_segmentation_masks(img, masks, alpha=0.5, colors="green")
    
    image_pil = T.ToPILImage()(drawn_image)
    image_pil.save(test_inference+"/{}.png".format(i))
    # image_pil.show()

dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxes', 'scores', 'labels'])
dict_keys(['boxe

In [16]:
5%3


2